In [12]:
#Hide code cells when saving as HTML
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

Toggle code

## Introducción

Este proyecto esta enfocado a predecir si una mujer es o no mayor a 23 años dado un conjunto de características presentado en una base de datos. 

La base de datos corresponde a las [*Interrupciones Legales de Embarazo*](https://datos.cdmx.gob.mx/explore/dataset/interrupcion-legal-del-embarazo) en la Ciudad de México. Esta base se obtuvo de la página de datos abiertos de la ciudad. 

Para contextualizar este proyecto, se consideran los siguientes supuestos:

  - El resultado de este proyecto será utilizado por una institución gubernamental, por lo tanto, los recursos son finitos, en particular se tiene un presupuesto limitado.
  - Se planea lanzar una campaña de servicios médicos para las mujeres que abortan. Se desea ofrecer servicios a bajo costo o gratuitos de salpingoclasia (esterilización permanente) y se quiere estimular a esta mujeres a participar en otros programas de bienestar social a los que pueden ser elegibles (por ejemplo, de madres solteras).
  


## Interrupción legal del embarazo CDMX

Es importante mencionar que la Interrupción del Embarazo solo era legal en la Ciudad de México, hasta hace unas pocas semanas, sin importar la causa o motivo. Pues si bien en otros estados también esta permitido, es sólo bajo ciertas circunstancias, principalmente relacionadas con delitos sexuales.

La interrupción del embarazo es legal en la Ciudad de México hasta las 12 semanas de gestación. Actualmente existen 13 clínicas en la ciudad que otorgan este servicio. La página de información de la CDMX menciona que los servicios se **proporcionan de manera legal, segura, confidencial y gratuita**. 

Los requisitos oficiales para obtener el servicio son los siguientes.

Requisitos para residentes de la Ciudad de México
+ Identificación oficial, en original y copia.
+ Comprobante de domicilio (último recibo de predial, luz, agua, gas, televisión de paga, teléfono fijo o servicio de internet), en original y copia.
+ Hoja de Gratuidad. Una trabajadora social te ayudará en caso de no tenerla.
+ Un acompañante con identificación oficial en original y copia.
+ De manera opcional en los hospitales pueden solicitarte: CURP y/o acta de nacimiento.

Requisitos para menores de edad:
+ Acta de Nacimiento en original y copia.
+ CURP (Puede imprimirse desde este sitio web)
+ Credencial o documento con fotografía reciente (credencial de la escuela o certificado de estudios) en original y copia.
+ Comprobante de domicilio en original y copia (último recibo de predial, luz, agua, gas, televisión de paga, teléfono fijo o servicio de internet).
+ Acudir acompañada por madre, padre, tutor o representante legal con identificación oficial y comprobante de domicilio, ambos en original y copia.

Requisitos para residentes de otros estados:
+ Original y copia de identificación oficial.
+ Comprobante de domicilio en original y copia.
+ Un acompañante con identificación oficial en original y copia.


In [13]:
#import python packages
import pandas as pd
import numpy as np
import pandas_profiling
from datetime import datetime, date, time, timedelta
# import calendar
from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, QuantileTransformer
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as dates
from matplotlib.gridspec import GridSpec

In [14]:
#import functions created in .py
import load_data 
import clean_data
import transform_data
import eda
import geda
import feature_selection

## Análisis EDA

In [15]:
##################################################  CARGAR BASE  #############################################################
#Update data file name
datafile_name = 'interrupcion-legal-del-embarazo.csv'
#Update path to data file

path_to_datafile = '~/Documents/Brenda/Proyecto/Data'
#path_to_datafile = '/home/bj/Documents/IntroDataScience/Proyecto/Data'
#path_to_datafile ='C:/Users/GZARAZUA/Desktop/MCD/MINERIA DE DATOS/Reporte aborto'

#we load the database
interrupcion_legal = load_data.load_df(datafile_name, path_to_datafile, ';')

La base original contiene información de interrupciones legales de embarazo (ILE) de 2016 a 2019.

Observamos que las variables son en su mayoría numericas (conteos) y algunas categóricas, que se relacionan en lo general con lo siguiente:
+ Fecha de registro: ano, mes, fingreso
+ Sitio donde se atiende al paciente y datos relacionados a la intervención: cve_hospital, autoref, desc_derechohab, consejería, h_ingreso, desc_servicio, p_consent, procile, s_complica, panticoncept
+ Características propias de la persona: edocivil, edad, nivel_edu, ocupacion, religion, parentezco, entidad, alc_o_municipio, menarca, fsexual, fmenstrua, sememb, nhijos, gesta, naborto, npartos, ncesarea, nile, anticonceptivo, c_num, motiles, p_semgest, p_diasgesta.


In [16]:
#mayor_23...


#Creamos la variable objetivo
clean_interrupcion_legal['23_o_mayor'] = np.where(clean_interrupcion_legal['edad'] >= 23, 1, 0)
clean_interrupcion_legal.head()

NameError: name 'clean_interrupcion_legal' is not defined

Después de la limpieza de la base de datos observamos cómo queda el registro de variables, que incluye las originales y otras tantes que usamos en forma auxiliar para analizar y/o separar los datos

In [ ]:
#variables that will contain the type of existing variables
[numeric_variables, categorical_variables, dates_variables, string_variables] = eda.info_type_of_vars(clean_interrupcion_legal)
#the classification of the variables is displayed
eda.print_info_type_of_vars(numeric_variables, categorical_variables, dates_variables, string_variables)

A continuación se muestra el dataprofiling de nuestras variables resultantes para una fácil identificación de estadísticos. 
Tambien se recomienda consultar el pandas profiling que este código genera

In [ ]:
eda.descriptive_stats_for_numeric_vars(clean_interrupcion_legal, numeric_variables)

La siguiente table indica las estadísticas de la variable categórica creada 'ocupacion2', que básicamente reduce el numero de categorías de la variable original 'ocupacion' rn, al tener este un no. considerable de opciones.

In [ ]:
#eda.descriptive_stats_for_categorical_vars(clean_interrupcion_legal, string_variables)
eda.descriptive_stats_for_categorical_vars(clean_interrupcion_legal, ['ocupacion2'])

Los estadísticos de 'edad_quintiles' dejan claro que la mayoría de las mujeres son jóvenes de entre 20 y 29 años

In [ ]:
eda.descriptive_stats_for_categorical_vars(clean_interrupcion_legal, categorical_variables)

Finalmente, se revisa la base como quedó después de todos los procedimientos descritos

In [ ]:
clean_interrupcion_legal.head()

# GEDA

In [ ]:
sns.set_style("darkgrid")
plt.figure(figsize = (20,10))
#plt.style.use('ggplot')
plot1 = sns.boxplot(y='edad', x='nile', data=clean_interrupcion_legal, palette='dark')
sns.stripplot(y='edad', x='nile', data=clean_interrupcion_legal, color="gray", jitter=0.2, size=2.5)
# Linea horizontal
plot1.axes.axhline(y = 23, ls='--', color='red')
plot1.axes.text(-0.8, 22.7, "23 años", color = 'red', fontsize=12)

plt.title('Distribución de la edad por número de ILE', fontsize='16')
plt.xlabel('Número de Interrupciones Legales (ILE)', fontsize='14')
plt.ylabel('Edad de la mujer', fontsize='14')

plt.show()

En la grafica anterior observamos como se comporta la edad de las mujeres que tuvieron 0,1,2,3,... interrupciones legales de embarazo, observando que apartir de la 3era interrupción las edades de mujeres tienden a concentrarse sólo entre los 20 y 40 años

In [ ]:
tabla_pivote = pd.pivot_table(clean_interrupcion_legal, index='edo_civil', columns='23_o_mayor', aggfunc='count')
tabla_pivote = tabla_pivote.reindex(['casada', 'union_libre', 'soltera', 'divorciada', 'separada','no_especificado'])
tabla_pivote['edad']

La tabla anterior señala que las mujeres más jóvenes que realizan un aborto legal son solteras, lo cual podría asumirse desde un inicio dado el ciclo natural de crecimiento de las personas y una tendencia social a extender la edad de contraer matrimonio

In [ ]:
ile_menor_23 = tabla_pivote['edad'][0]/sum(tabla_pivote['edad'][0])*100
ile_mayor_23 = tabla_pivote['edad'][1]/sum(tabla_pivote['edad'][1])*100

In [ ]:
df_aux = pd.DataFrame(tabla_pivote['edad'][0]/sum(tabla_pivote['edad'][0])*100)
df_aux = pd.concat([df_aux, tabla_pivote['edad'][1]/sum(tabla_pivote['edad'][1])*100])
df_aux['23_0_mas'] = [0,0,0,0,0,0,1,1,1,1,1,1]
df_aux['edo_civil1'] = ['casada', 'union_libre', 'soltera', 'divorciada', 'separada','no_especificado', 'casada', 'union_libre', 'soltera', 'divorciada', 'separada','no_especificado']
df_aux.columns = ['Porcentaje', '23_o_mayor', 'edo_civil1']
df_aux.index = df_aux.reset_index(level=0, drop=True)
df_aux.index = [0,1,2,3,4,5,6,7,8,9,10,11]
#
g = sns.catplot(y="Porcentaje", x='edo_civil1', hue='23_o_mayor', kind="bar", data=df_aux, palette=['gray', 'blue'], height=6, aspect=1.3, legend_out=False)
g.ax.set_yticks(np.arange(0,110,10), minor=True)

leg = g.axes.flat[0].get_legend()
leg.set_title('')
new_labels=['Menores a 23 años', '23 años o más']
for t, l in zip(leg.texts, new_labels): t.set_text(l)
    
plt.ylabel("")
plt.xlabel("")
plt.title("Porcentaje de mujeres que recibieron una ILE por Estado Civil")
plt.show()

Corroboranado la información de la tabla anterior, el saber de una mujer que se practica una interrución legal de embarazo, que es soltera, sugiere que es joven

In [ ]:
# df_aux = clean_interrupcion_legal[clean_interrupcion_legal.edo_civil.isin(['casada', 'soltera','union_libre'])][['edo_civil', 'nile', 'nhijos', '23_o_mayor']]
# pd.pivot_table(df_aux, index=['edo_civil', 'nhijos'], columns='nile', aggfunc='count')
# #sns.lmplot( y="nile", x="nhijos", data=df_aux, fit_reg=False, hue='edo_civil', legend=True)

In [ ]:
# df_aux = clean_interrupcion_legal.groupby(['entidad', '23_o_mayor']).count()['mes'].unstack(level=1).reset_index().sort_values(by=1, ascending=False)
# df_aux[0]= df_aux[0]/sum(df_aux[0])*100
# df_aux[1]= df_aux[1]/sum(df_aux[1])*100
# df_aux.columns = ['entidad', 'menor', 'mayor']

# df_aux2 = df_aux[0:2]
# df_aux3 = df_aux[2:]
# df_aux2 = df_aux2.append(pd.Series(['otras_entidades',sum(df_aux3['menor']), sum(df_aux3['mayor'])], index=df_aux2.columns), ignore_index=True)

# #labels = df_aux2['entidad']
# labels = ['Ciudad \n de México', 'Estado \n de Mexico', 'Otras \n entidades']
# x = np.arange(len(labels))  # the label locations
# width = 0.35  # the width of the bars



# fig = plt.figure(figsize=(20, 5))
# gs = GridSpec(nrows=1, ncols=5)

# ax1 = fig.add_subplot(gs[0,1])
# ax1.bar(x - width/2, df_aux2['menor'], width, color='gray', label='Menores de 23 años')
# ax1.bar(x + width/2, df_aux2['mayor'], width, color='blue', label='23 años o más')
# ax1.set_xticks(x)
# ax1.set_xticklabels(labels)
# ax1.set_yticks(np.arange(0,110,10), minor=True)
# ax1.set_title("Top de Entidad de Residencia")
# ax1.legend(loc='upper right')

# ax2 = fig.add_subplot(gs[0,2:])
# my_size=np.where(df_aux3['mayor']>df_aux3['menor'], 75, 30)
# my_range = df_aux3['entidad']
# ax2.vlines(x = my_range, ymin=df_aux3['menor'], ymax=df_aux3['mayor'], color='black', alpha=0.4)
# ax2.axvline(x = 'queretaro', color='red', ls='--', linewidth=0.5)
# ax2.scatter(my_range, df_aux3['menor'], color='gray', s=my_size, alpha=1, label='Menores de 23 años')
# ax2.scatter(my_range, df_aux3['mayor'], color='blue', s=my_size, alpha=1 , label='23 años o más')
# ax2.set_xticks(np.arange(len(my_range)))
# ax2.set_xticklabels(my_range, rotation=90, horizontalalignment='right')
# ax2.set_title("Desglose de Otras entidades federativas")
# ax2.legend(loc='upper right')

# fig.suptitle("Porcentaje de ILE segun la entidad donde residen")
# #format_axes(fig)

# plt.show()

In [ ]:
pd.options.display.float_format = '{:,}'.format
pivot_aux = pd.pivot_table(clean_interrupcion_legal, index='escolaridad', columns='edo_civil', aggfunc='count')['mes']
pivot_aux = pivot_aux.reindex(['posgrado', 'superior', 'media_superior', 'secundaria', 'primaria','otra', 'ninguno', 'no_especificado' ])
pivot_aux = pivot_aux[['casada', 'union_libre', 'soltera', 'divorciada', 'separada','no_especificado']]
pivot_aux
#sns.heatmap(pivot_aux)
#pd.pivot_table(clean_interrupcion_legal, index='escolaridad', columns=['23_o_mayor', 'edo_civil'], aggfunc='count')['mes']

De esta muestra que la mayoría de las mujeres comparte la característica de estar soltera y tener como grado máximo de estudios educación media superior o secundaria.
La mayoría de las mujeres que viven en unión libre comparte el mismo nivel máximo de estudios indicado anterioremente

In [ ]:
pivot_aux = pd.pivot_table(clean_interrupcion_legal, index='escolaridad', columns='ocupacion2', aggfunc='count')['mes']
pivot_aux = pivot_aux.reindex(['posgrado', 'superior', 'media_superior', 'secundaria', 'primaria','otra', 'ninguno', 'no_especificado' ])
pivot_aux 

Esta otra tabla muestra que la mayoría de mujeres tiene el nivel máximo de estudios de secundaria o media superior, y que existe un grupo considerable de mujers que depende monetariamente de terceros, pues el grupo de estudiantes y amas de casa es significativo. La mayoría de mujeres se caracteriza por ser empleadas.

In [ ]:
pivot_aux = pd.pivot_table(clean_interrupcion_legal, index='edo_civil', columns='ocupacion2', aggfunc='count')['mes']
pivot_aux = pivot_aux.reindex(['casada', 'union_libre', 'soltera', 'divorciada', 'separada','no_especificado'])
pivot_aux 

La mayoría de las mujeres son empleadas/estudiantes y solteras, aunque existen otros grupos significativos como las mujeres que se dicen vivir en unión libre, que en su mayoría son amas de casa o empleadas.

In [ ]:
geda.plot_by_pairs_grid(clean_interrupcion_legal, ['edad','naborto'])

In [ ]:
#geda.plot_by_pairs_grid(clean_interrupcion_legal, ['menarca','edad'])

Aunque la mayoría de las mujeres no indicó haber tenido alguna interrupción legal de embarazo previamente, se observa que hay mujeres que de entre 20 y 40 años que dicen haber tenido de entre 1 a 4, lo cual podría ser un indicativo de mujeres que se han practicado más de una vez un aborto, pues los datos corresponden a 3 diferentes años (2016-2019)

In [ ]:
geda.barplot_by_category(clean_interrupcion_legal, 'edad', 'ocupacion2')

Una suposición natural indica que que la mayoría de mujeres estudiantes son jóvenes, lo cual se corrobora con la gráfica anterior. El resto de ocupaciones indican una poblacion que supera los 23 años, lo que podría ser un indicativo en la importancia de esta variables en la predicción de la edad

In [ ]:
geda.barplot_by_category(clean_interrupcion_legal, 'edad', 'edocivil_descripcion')

Pareciera existir una relacion entre el edo civil y la edad de las mujeres que practican interrupción del embarazo, pues el promedio de edad es diferente para cada categoría. No obstante lo anterior, se tendría que analizar más a fondo la distribución de los datos, pues la medida de promedio es muy sensible a outlayers

In [ ]:
####################################  PANDAS PROFILING  #########################################
# #Imprime a pantalla
# clean_interrupcion_legal.profile_report(correlations={"cramers": False})

# #Imprime a archivo

# profile = clean_interrupcion_legal.profile_report(title='Interrupcion legal del embarazo', plot={'histogram': {'bins': 8}}, correlations={"cramers": False})
# profile.to_file(output_file="Interrupcion legal del embarazo_output.html")

In [ ]:
# #guardamos la base de datos resultante a un .csv
# clean_interrupcion_legal.to_csv('clean_interrupcion_legal.csv', sep=';')

# Insights más importantes de este conjunto de datos

+ La mayoría de mujeres poseen una educación máxima de media superior o secundaria, con estudios terminados o incompletos. La educación se podría considerar como un factor relevante en la toma de decisiones de las personas, que abarca la planeación a futuro

+ Las mayoría de mujeres indicó ser solteras y de estas la mayoría son jóvenes. El hecho de tener algún compromiso para con una pareja o hijos pareciera otorgar mayor libertad para tomar una decisión sobre la interrupción del embarazo.

+ Considerando que la base contiene la información de diversos años, y que existen mujeres que se han practicado más de un aborto, podría indicar que hay mujeres que hacen uso de la interrupción legal de embarazo varias veces. Esto podría sesgar las predicciones dado que podríamos tener el caso de registrar a una misma persona varias veces

+ Las semanas/dias de gestación guardan una aparente fuerte correlación lineal positiva con la edad. Esto podría ser un indicativo del tiempo que se dan las personas para una toma de decisión, donde las personas más jovenes tienen a tomar la decisión en forma más temprana

+ Dado que el objetivo planteado es predecir la edad de las mujeres dadas sus características, se han descartado algunas variables como las relacionadas a eventos posteriores a la intervención, cuya consideración podría incidir en hacer data leaking. Existen otras variables que aportan mucha información al prevalecer una categoría o valor como altamente representativo, como puede ser la religión. Existen variables que a primera vista podrían sugerir un alto nivel predictivo, como son motiles (motivos por los cuales se realiza la intervención) o parentesco (persona que acompaña a la paciente), sin embargo se detecta que las respuestas a estos concepto no son de mucha ayuda. En motile la mayoría indica tal cual interrupción voluntaria, lo cual no aporta nada). Aunque se sabe que para cualquier intervención se pide se vaya acompañada, la mayoría de las personas no reporta información sobre el parentesco del acompañante

## Separacion de datos de *entrenamiento* y *prueba*

La partición de la base en el conjunto de entrenamiento y de prueba, se realizo usando temporal cross validation, pues los eventos considerados (ILE) están intrínsicamente relacionados con el tiempo. Además, las características de la observaciones en la base, y de la población objetivo -mujeres de 23 años o más- en particular, son cambiantes a lo largo del tiempo.

La muestra de entrenamiento se selecciono desde la fecha más antigua del primer registro (2016-01-01) hasta la fecha límite elegida, en este caso 2018-07-01. Así, la muestra de prueba fueron las observaciones a partir de 2018-08-01 y hasta la última fecha disponible en la base (2019-07-01).

Esta fecha se eligio para tener al menos un año en la muestra de prueba, pensando que puede haber estacionalidad a lo largo de un año. Además, con esta fecha se obtiene que el 74% de los datos se encuentran en la muestra de entrenamiento y el restante en la muestra de prueba.

In [ ]:
#Lo deje hasta julio de 2018 poara tener un anio de prueba
mask = (base_interrupcion_legal['ano_mes_ile']<='2018-07-01')
entrenamiento = base_interrupcion_legal[mask]
prueba = base_interrupcion_legal[~mask]
print('No. observaciones en entre: ', entrenamiento.shape[0], ' que representa el ', round(entrenamiento.shape[0]/base_interrupcion_legal.shape[0]*100,0),'%')
print('No. observaciones en prueb: ', prueba.shape[0], ' que representa el ', round(prueba.shape[0]/base_interrupcion_legal.shape[0]*100,0),'%')

In [ ]:
y_entrenamiento = entrenamiento['23_o_mayor']
x_entrenamiento = pd.DataFrame(entrenamiento.drop(['23_o_mayor', 'ano_mes_ile'], axis = 1))

y_prueba = prueba['23_o_mayor']
x_prueba = pd.DataFrame(prueba.drop(['23_o_mayor', 'ano_mes_ile'], axis = 1))

# Imputacion de variables

Se realizaron imputaciones a las siguientes variables de la muestra de entrenamiento:

| Variable | % missings | Valor de imputación | Justificacion |
|----------|------------|---------------------|---------------|
| edo_civil | 0.32 | moda (soltera) | 55% de los datos corresponden a esa categoría |
|escolaridad | 2.54 | moda (media_superior) | 45% de los datos corresponden a esa categoría |
|menarca| 5.05 | moda (12 años) | %27 de los datos corresponden a esa categoría |
|anticonceptivo|12.50|constante (ninguno)| 53% de los datos corresponden a esa categoría|
|ocupacion| 15.91 | constante (no_especificado)| Nueva categoria para aislar el efecto de los valores faltantes|
|nhijos| 5.32 | mediana | 40% de los datos corresponden a esa categoría y la media y la mediana de los datos es muy similar|
|naborto| 6.47 | mediana |80% de los datos corresponden a esa categoría y la media y la mediana de los datos es muy similar|
|nparto|6.16| mediana | 56% de los datos corresponden a esa categoría y la media y la mediana de los datos es muy similar|
|ncesarrea|6.64| mediana | 70% de los datos corresponden a esa categoría y la media y la mediana de los datos es muy similar|
|nile|26.4| mediana | 63% de los datos corresponden a esa categoría y la media y la mediana de los datos es muy similar|

>> Porcentajes respecto a la muestra de entrenamiento

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, QuantileTransformer
from sklearn.impute import SimpleImputer

In [ ]:
#Imputacion para base de entrenamiento

df = x_entrenamiento

var = 'edo_civil'
aux = df[[var]]
aux_imputer_edo_civil = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
aux_imputer_edo_civil.fit(aux)
aux_imputed_edo_civil = aux_imputer_edo_civil.transform(aux)
df[var] = aux_imputed_edo_civil

var = 'escolaridad'
aux = df[[var]]
aux_imputer_escolaridad = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
aux_imputer_escolaridad.fit(aux)
aux_imputed_escolaridad = aux_imputer_escolaridad.transform(aux)
df[var] = aux_imputed_escolaridad

var = 'menarca'
aux = df[[var]]
aux_imputer_menarca = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
aux_imputer_menarca.fit(aux)
aux_imputed_menarca = aux_imputer_menarca.transform(aux)
df[var] = aux_imputed_menarca

#var = 'p_diasgesta'
#aux = df[[var]]
#aux_imputer_p_diasgesta = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
#aux_imputer_p_diasgesta.fit(aux)
#aux_imputed_p_diasgesta = aux_imputer_p_diasgesta.transform(aux)
#df[var] = aux_imputed_p_diasgesta


var = 'anticonceptivo2'
aux = df[[var]]
aux_imputer_anticonceptivo2 = SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="ninguno")
aux_imputer_anticonceptivo2.fit(aux)
aux_imputed_anticonceptivo2 = aux_imputer_anticonceptivo2.transform(aux)
df[var]=aux_imputed_anticonceptivo2


var = 'ocupacion2'
aux = df[[var]]
aux_imputer_ocupacion2 = SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="no_especificado")
aux_imputer_ocupacion2.fit(aux)
aux_imputed_ocupacion2 = aux_imputer_ocupacion2.transform(aux)
df[var]=aux_imputed_ocupacion2

# var = 'p_semgest'
# aux = df[[var]]
# aux_imputer_p_semgest = SimpleImputer(missing_values=np.nan, strategy="median")
# aux_imputer_p_semgest.fit(aux)
# aux_imputed_p_semgest = aux_imputer_p_semgest.transform(aux)
# df[var]=aux_imputed_p_semgest

var = 'nhijos'
aux = df[[var]]
aux_imputer_nhijos = SimpleImputer(missing_values=np.nan, strategy="median")
aux_imputer_nhijos.fit(aux)
aux_imputed_nhijos = aux_imputer_nhijos.transform(aux)
df[var]=aux_imputed_nhijos

var = 'naborto'
aux = df[[var]]
aux_imputer_naborto = SimpleImputer(missing_values=np.nan, strategy="median")
aux_imputer_naborto.fit(aux)
aux_imputed_naborto = aux_imputer_naborto.transform(aux)
df[var]=aux_imputed_naborto

var = 'npartos'
aux = df[[var]]
aux_imputer_nparto = SimpleImputer(missing_values=np.nan, strategy="median")
aux_imputer_nparto.fit(aux)
aux_imputed_nparto = aux_imputer_nparto.transform(aux)
df[var]=aux_imputed_nparto

var = 'ncesarea'
aux = df[[var]]
aux_imputer_ncesarea = SimpleImputer(missing_values=np.nan, strategy="median")
aux_imputer_ncesarea.fit(aux)
aux_imputed_ncesarea = aux_imputer_ncesarea.transform(aux)
df[var]=aux_imputed_ncesarea

var = 'nile'
aux = df[[var]]
aux_imputer_nile = SimpleImputer(missing_values=np.nan, strategy="median")
aux_imputer_nile.fit(aux)
aux_imputed_nile = aux_imputer_nile.transform(aux)
df[var]=aux_imputed_nile



# Feature Engineering

## Random forest para parametros


Algunas variables categoricas se recategorizaron para agrupar etiquetas similares y disminuir el número de categorias.

Originalmente se tienen 36 variables, y aunque no es un gran número de variables en términos de los modelos de aprendizaje de máquina, es importante hacer la selección de variables para eliminar aquellas ue son espurias y que pudieran estar incorporando ruido al modelo y por lo tanto, disminuir su desempeño.

En base al EDA se incluyeron 10 variables (edo_civil, escolaridad, ocupacion2, menarca, anticonceptivo2, nhijos, naborto, npartos, ncesarea, nile) para correr un modelo de Random Forest para determinar el nivel de importancia de las variables.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [ ]:
N_ARBOLES = 10000  #The number of trees in the forest
SEMILLA = 104 # the seed used by the random number generator

#Create a Gaussian Classifier
clf = RandomForestClassifier(n_estimators = N_ARBOLES, random_state = SEMILLA, n_jobs=-1)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_entrenamiento, y_entrenamiento)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

feature_imp = pd.Series(clf.feature_importances_, index=x_entrenamiento.columns).sort_values(ascending=False)
feature_imp

In [ ]:
sns.set_style("darkgrid")
pp = sns.cubehelix_palette(30, start=.1, rot=-.75, reverse=True)
fig = plt.figure(figsize=(8, 12))
#Visualizing Important Features
g = sns.barplot(x=feature_imp, y=feature_imp.index, palette=pp)
# Add labels to your graph
#g.axes.axhline(y = 'naborto', ls='--', color='red')
plt.xlabel('Score')
plt.ylabel('Variables')
plt.title("Vizualizacion del score de importancia de las variables")
plt.show()


El Random Forest se corrio con 100, 1,000 y 10,000 árboles y se determino un *threshold* para la importancia, de $0.046$. Se observo que,todos los casos tuvieron el mismo orden de relevancia para las variables.

Nos quedamos con las siguientes variables:
- menarca
- nhijos
- ocupacion2_estudiante
- npartos
- escolaridad_superior
- ocupacion2_empleada
- ncesarea
- naborto

In [ ]:
#Para entrenamiento
x_entrenamiento = x_entrenamiento[['menarca', 'nhijos', 'ocupacion2_estudiante', 'npartos', 'escolaridad_superior', 'ocupacion2_empleada', 'ncesarea', 'naborto']]


## Hiperparámetros (Magic Loop)


El ajuste de los parámetros se hizo para tres modelos distintos: Decision Tree, Random Forest y XGBoost. 

Cada modelo se corrio considerando los siguientes rangos para los parámetros:}


|Modelo| Parametros|# de modelos| Tiempo de ejecución (mins)| Mejor score |
|------|-----------|------------|---------------------------|-------------|
|Decision Tree| max_depth: [1,5,10,20,50,100], |  1,440|  7 | 0.82 |
|             | min_samples_split: [2,5,10], | | | |
|             | criterion:['gini', 'entropy'], | | | |
|             | splitter:['random', 'best'], | | | |
|             | presort:[False, True] | | | |
|Random Forest| n_estimators: [1,10,100,1000], | 1,440| 189 | 0.83|
|             | max_depth: [1,5,10,20,50,100], | | | |
|             | max_features: ['sqrt','log2'], | | | |
|             | min_samples_split: [2,5,10] | | | |
|XGBoost| max_depth: [1,5,10,20,50,100], | 540 | 120| 0.83 |
|       | *min_child_weight*: [2,5,10], | | | |
|       | *learning_rate* : [0.01,0.050.1]   | | | |         
|       | objective:['squaredlogerror', 'binary:logistic', 'binary:hinge'] | | | |


Para cada conjunto de parámetros, el modelo uso *temporal cross validation* con *10 folds*.

Probando 3 modelos diferentes con distinto hiperparametros, donde seleccionamos el mejor modelo con base en el score F1, que relaciona las metricas *precision* y *recall*.

Lo anterior considerando que el uso de estos resultados podrá usarse para promover alguna politica pública de salud, donde los recursos son finitos e interesa seleccionar con precison a las personas objetivo.


| Modelo | Color   | Precio |
| ------ |---------| ------:|
| Globe  | Negro   | 99€    |
| Scala  | Azul    | 199€   |
| Palais | Granate | 399€   |


In [ ]:
from sklearn.model_selection import TimeSeriesSplit #Para split temporal
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, auc, roc_curve
from sklearn import tree
import xgboost as xgb


### 1. Random Forest

In [ ]:
#ocuparemos un RF
classifier = RandomForestClassifier()

##separando en train, test
#X_train, X_test, y_train, y_test = train_test_split(X, y)

#definicion de los hiperparametros que queremos probar
hyper_param_grid1 = {'n_estimators': [1,10,100,1000], 
                    'max_depth': [1,5,10,20,50,100], 
                    'max_features': ['sqrt','log2'],
                    'min_samples_split': [2,5,10]}

#ocupemos grid search!, el verbose permite hacer debugging ... el 3 nos permitirá ver los mensajes de 3 trials de los 10 
grid_search = GridSearchCV(classifier, 
                           hyper_param_grid1, 
                           scoring = 'f1',
                           cv = cv, 
                           n_jobs = -1,
                           verbose = 3)
grid_search.fit(x_entrenamiento, y_entrenamiento)

#de los valores posibles que pusimos en el grid, cuáles fueron los mejores
grid_search.best_params_

#mejor score asociado a los modelos generados con los diferentes hiperparametros
#corresponde al promedio de los scores generados con los cv
grid_search.best_score_

In [ ]:
cv_results_forest = pd.DataFrame(grid_search.cv_results_)
cv_results_forest.head()
#Save to file
cv_results_forest.to_csv(index=False)

### 2. Arbol

In [ ]:
#ocuparemos un RF
classifier = tree.DecisionTreeClassifier()

##separando en train, test
#X_train, X_test, y_train, y_test = train_test_split(X, y)

#definicion de los hiperparametros que queremos probar
hyper_param_grid2 = {'max_depth': [1,5,10,20,50,100], 
                     'min_samples_split': [2,5,10],
                    'criterion':['gini', 'entropy'],
                    'splitter':['random', 'best'],
                    'presort':[False, True]}

#ocupemos grid search!, el verbose permite hacer debugging ... el 3 nos permitirá ver los mensajes de 3 trials de los 10 
grid_search = GridSearchCV(classifier, 
                           hyper_param_grid2, 
                           scoring = 'f1',
                           cv = cv, 
                           n_jobs = -1,
                           verbose = 3)
grid_search.fit(x_entrenamiento, y_entrenamiento)

#de los valores posibles que pusimos en el grid, cuáles fueron los mejores
grid_search.best_params_

#mejor score asociado a los modelos generados con los diferentes hiperparametros
#corresponde al promedio de los scores generados con los cv
grid_search.best_score_

In [ ]:
cv_results_arbol = pd.DataFrame(grid_search.cv_results_)
cv_results_arbol.head()
#
cv_results_arbol.to_csv(index=False)

### 3. XGBoost

In [ ]:
#ocuparemos un RF
classifier = xgb.XGBClassifier()

##separando en train, test
#X_train, X_test, y_train, y_test = train_test_split(X, y)

#definicion de los hiperparametros que queremos probar
hyper_param_grid = {'max_depth': [1,5,10,20,50,100],
                    'min_samples_split': [2,5,10],
                    'booster':['gbtree', 'gblinear', 'dart']
                    }

#ocupemos grid search!, el verbose permite hacer debugging ... el 3 nos permitirá ver los mensajes de 3 trials de los 10 
grid_search = GridSearchCV(classifier, 
                           hyper_param_grid, 
                           scoring = 'f1',
                           cv = cv, 
                           n_jobs = -1,
                           verbose = 3)
grid_search.fit(x_entrenamiento, y_entrenamiento)

#de los valores posibles que pusimos en el grid, cuáles fueron los mejores
grid_search.best_params_

#mejor score asociado a los modelos generados con los diferentes hiperparametros
#corresponde al promedio de los scores generados con los cv
grid_search.best_score_

In [ ]:
cv_results_XGBoost = pd.DataFrame(grid_search.cv_results_)
cv_results_XGBoost.head()
#
cv_results_XGBoost.to_csv(index=False)

Juntamos los resultados de todos los modelos

In [ ]:
results_all = pd.concat([cv_results_forest, cv_results_arbol, cv_results_XGBoost], axis=0, sort='False').sort_values(by='rank_test_score', ascending=True).head()

## mejor configuración de hiperparámetros de acuerdo a nuestra métrica
results_all.iloc[0,:].params

In [ ]:
#cv_results_forest
#cv_results_arbol
cv_results_XGBoost

El mejor modelo obtenido fue un XGBoost, y el mejor conjunto de hiperparametros se muestra a continuación:

Con base en el mean-best-score se tomó el mejor modelo, ya que incluye el promedio de las 10 corridas de cross-validation

De los modelos que se corrieron, se obtuvieron los siguientes tiempos:
- Random Forest (1,440 modelos): 75.6 min
- Arbol (1,440 modelos): 1.6 min
- XGBoost (540 modelos): 62.2 min


## Seleccion de modelo  y aplicacion en muestra de prueba

In [ ]:
#Imputacion para base de prueba

df = x_prueba

var = 'edo_civil'
aux = df[[var]]
aux_imputed_edo_civil = aux_imputer_edo_civil.transform(aux)
df[var] = aux_imputed_edo_civil

var = 'escolaridad'
aux = df[[var]]
aux_imputed_escolaridad = aux_imputer_escolaridad.transform(aux)
df[var] = aux_imputed_escolaridad

var = 'menarca'
aux = df[[var]]
aux_imputed_menarca = aux_imputer_menarca.transform(aux)
df[var] = aux_imputed_menarca

var = 'anticonceptivo2'
aux = df[[var]]
aux_imputed_anticonceptivo2 = aux_imputer_anticonceptivo2.transform(aux)
df[var]=aux_imputed_anticonceptivo2


var = 'ocupacion2'
aux = df[[var]]
aux_imputed_ocupacion2 = aux_imputer_ocupacion2.transform(aux)
df[var]=aux_imputed_ocupacion2

var = 'nhijos'
aux = df[[var]]
aux_imputed_nhijos = aux_imputer_nhijos.transform(aux)
df[var]=aux_imputed_nhijos

var = 'naborto'
aux = df[[var]]
aux_imputed_naborto = aux_imputer_naborto.transform(aux)
df[var]=aux_imputed_naborto

var = 'npartos'
aux = df[[var]]
aux_imputed_nparto = aux_imputer_nparto.transform(aux)
df[var]=aux_imputed_nparto

var = 'ncesarea'
aux = df[[var]]
aux_imputed_ncesarea = aux_imputer_ncesarea.transform(aux)
df[var]=aux_imputed_ncesarea

var = 'nile'
aux = df[[var]]
aux_imputed_nile = aux_imputer_nile.transform(aux)
df[var]=aux_imputed_nile



In [ ]:
### One -hot encoder Para Prueba
dummies = feature_selection.transforma_cat_dummies(x_prueba, string_variables, False)
x_prueba = pd.concat([x_prueba, dummies], axis = 1)
x_prueba = x_prueba.drop(string_variables, axis = 1)


In [ ]:
# Seleccion de variables Para entrenamiento
x_prueba = x_prueba[['menarca', 'nhijos', 'ocupacion2_estudiante', 'npartos', 'escolaridad_superior', 'ocupacion2_empleada', 'ncesarea', 'naborto']]

In [ ]:
x_prueba

tomado de [aqui](https://jessesw.com/XG-Boost/)

In [ ]:
# Definicion de los parametros del modelo
# specify parameters via map
param_finales = {'max_depth':1, 'min_samples_split': 10, 'booster':'gbtree'}

num_round = 500

xgdmat = xgb.DMatrix(x_entrenamiento, y_entrenamiento)

best_model = xgb.train(param_finales, xgdmat, num_round)


In [ ]:
#we can then plot our feature importances using a built-in method. This is similar to the feature importances found in sklearn.
xgb.plot_importance(best_model)

Prediccion para la muestra de prueba

In [ ]:
testdmat = xgb.DMatrix(x_prueba)
y_predict = best_model.predict(testdmat)
y_predict

In [ ]:
# Convertimos a etiquetas 0/1
THRESHOLD = 0.5

y_predict[y_predict > THRESHOLD] = 1
y_predict[y_predict <= THRESHOLD] = 0
y_predict


Acuracy en la muestra de prueba

In [ ]:
accuracy_score(y_predict, y_prueba)

In [ ]:
recall_score(y_prueba, y_predict, average='weighted')

In [ ]:
f1_score(y_prueba, y_predict, average='weighted')

In [ ]:
# ROC curve
fpr, tpr, thresholds = metrics.roc_curve(y_prueba, y_predict)
roc_auc = auc(fpr, tpr)

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([-0.02, 1.0])
plt.ylim([0.0, 1.02])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
#Confusion matrix

print(confusion_matrix(y_prueba, y_predict))

La matriz de confusion indica:
verdaderos positivos (mujeres clasificadas como >=23 siendo que es >=23)=20%
falsos verdaderos (mujeres clasificadas como <23 siendo que son >=23)=15%
falso negativos (mujeres clasificadas como >=23 siendo que son <23)=8%
verdadero negativos (mujeres clasificadas como <23 siendo que son <23)=57%
Lo anterior indica que solo 23% de los datos fueron mal clasificados


Dado que consideramos que el uso de este modelo podria ser usado para promover politicas publicas de salud, asumimos que los recursos son finitos y por tanto se busca optimizar los mismos reduciendo al maximo los falsos positivos

Precison at k:
Podemos considerar que se tienen recursos finitos, por ejemplo un presupuesto limite, por lo tanto solo se puede contratar k medicos para realizar los procedimientos o k trabajadoras sociales en educacion sexual. En el caso de se trabaje con voluntarios que no estan sujetos a una restriccion presupuestaria, podriamos tomar a k como los consultorios fisicos en los que se puede trabajar.


# Implicaciones éticas:

+ Planteamiento del objetivo: 
Se desconoce la finalidad del objeto de predecir si una mujer que se practica una ILE tiene 23 años o más, es decir, el uso que se le dará los analisis y modelos obtenidos. Por ejemplo, si se usa para promover políticas públicas (otorgamiento o restricción de beneficios) o para ofrecer alguna alternativa por parte del sector privado, la métrica de medición podría ser enfocada a optimizar recuros u optimizar la detección de mujeres de 23 años o más. Así mismo, las implicaciones éticas podrían variar según sea el uso que se da a los resultados.

+ EDA: 
Los datos obtenidos provienen de los registros de mujeres que se practican ILE en la CDMX, donde la opción de ejercerla es libre (por cualquier motivo que considere la persona). Existen estados donde la ILE es legal pero sólo bajo ciertos motivos (como violación, que ponga en peligro de la mujer, etc). Se observo que hay mujeres de otros entidades que vienen a la CDMX para practicarse el ILE, eso podría sesgar la motivación o tipo de personas que se practican el ILE en la CDMX.

+ Imputación de datos:
La corrección y/o imputación de valores sugiere que la forma de recabar los datos podría no ser del todo confiable o fidedigna, pues variables como el parentesco con el acompañante, podrían sugerir que las pacientes o recolectores de datos no proporcionan o reportan todos los datos correctamente (existen muchas mujers menores de edad que no reportan parentescon del acompañante, desconociendo si no hubo acompañante o deliverdamente no se reporto).

Para efectos de poder correr los modelos, para algunas de las variables se sustituyo los NAs con la mediana, sin embargo parala variable nile (no. de ILE) había 20% de faltantes, lo cual podría sesgar la interpretabilidad o efecto en el modelo, si se toma esta variable

+ Feature engineering:
Para variables categoricas como ocupacion, anticonceptivo, nivle_edu y edocivil_descripcion se probó reducir el numero de categorías que tenían para logara incrementar el valor predictivo, no obstante implica pérdida de información que aporta la variable original. Para el entendimiento del modelo o interpretación, la reducción de información podría derrivar en generalidades imprecisas.

+ Selección de variables:
Para lograr meter variables categóricas a los modelos, se crearon variables dummy, y posterioremente se decidió qué variables se quedarían en el modelo. El seleccionar sólo algunas dummies de una variable categorica (sin abarcar todos los valores que toma), tiene un efecto similar al planteado en feature engineering (con la reducción de categorias).

+ Selección de modelo:
La selección implicó el probar muchos parámetros para los diferentes modelos probados, escogiendo el que minimizó el score F1, sin indicar las implicaciones y/o diferencias entre uno u otro.
